#  Anomaly detection in cellular networks

## 1. Introduction

The purpose of this notebook is to solve a anomaly detection problem proposed as a competition in the Kaggle InClass platform.

## 2. Problem description

### Context:

Traditionally, the design of a cellular network focuses on the optimization of energy and resources that guarantees a smooth operation even during peak hours (i.e. periods with higher traffic load). 
However, this implies that cells are most of the time overprovisioned of radio resources. 
Next generation cellular networks ask for a dynamic management and configuration in order to adapt to the varying user demands in the most efficient way with regards to energy savings and utilization of frequency resources. 
If the network operator were capable of anticipating to those variations in the users’ traffic demands, a more efficient management of the scarce (and expensive) network resources would be possible.
Current research in mobile networks looks upon Machine Learning (ML) techniques to help manage those resources. 
In this case, you will explore the possibilities of ML to detect abnormal behaviors in the utilization of the network that would motivate a change in the configuration of the base station.


### Objective

The objective of the network optimization team is to analyze traces of past activity, which will be used to train an ML system capable of classifying samples of current activity as:
 - 0 (normal): current activity corresponds to normal behavior of any working day and. Therefore, no re-configuration or redistribution of resources is needed.
 - 1 (unusual): current activity slightly differs from the behavior usually observed for that time of the day (e.g. due to a strike, demonstration, sports event, etc.), which should trigger a reconfiguration of the base station.

### Dataset

The dataset has been obtained from a real LTE deployment. During two weeks, different metrics were gathered from a set of 10 base stations, each having a different number of cells, every 15 minutes. 

The dataset is provided in the form of a csv file, where each row corresponds to a sample obtained from one particular cell at a certain time. Each data example contains the following features:

 - Time : hour of the day (in the format hh:mm) when the sample was generated.
 - CellName1: text string used to uniquely identify the cell that generated the current sample. CellName is in the form xαLTE, where x identifies the base station, and α the cell within that base station (see the example in the right figure).
 - PRBUsageUL and PRBUsageDL: level of resource utilization in that cell measured as the portion of Physical Radio Blocks (PRB) that were in use (%) in the previous 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - meanThrDL and meanThrUL: average carried traffic (in Mbps) during the past 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - maxThrDL and maxThrUL: maximum carried traffic (in Mbps) measured in the last 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - meanUEDL and meanUEUL: average number of user equipment (UE) devices that were simultaneously active during the last 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - maxUEDL and maxUEUL: maximum number of user equipment (UE) devices that were simultaneously active during the last 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - maxUE_UL+DL: maximum number of user equipment (UE) devices that were active simultaneously in the last 15 minutes, regardless of UL and DL.
 - Unusual: labels for supervised learning. A value of 0 determines that the sample corresponds to normal operation, a value of 1 identifies unusual behavior.

## Libraries

In [67]:
import os
import sys
import random
random.seed(888) #set seed for reproducibility
from zipfile import ZipFile
from IPython.display import Image


#Analysis
import pyspark
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    print('WARN: Something wrong with pyspark library. Please check configuration settings!')
    
#Feature Engineering
from pyspark.sql.functions import col, when, lit, array, explode, rand
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
    
# Reloads functions each time so you can edit a script and not need to restart the kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Helpers

In [2]:
def get_root_dir(src:str, max_nest:int) -> str:
    '''
    Specify paths and appending directories with relevant python source code.
    '''
    root_dir = os.curdir
    nest = 0
    while src not in os.listdir(root_dir) and nest < max_nest:
        root_dir = os.path.join(os.pardir, root_dir)     # Look up the directory structure for a src directory
        nest += 1
        
    # If you don't find the src directory, the root directory is this directory
    root_dir = os.path.abspath(root_dir) if nest < max_nest else os.path.abspath(
    os.curdir)
    
    return root_dir

def set_src(root_dir:str, src:str) -> str:
    '''
     Get the source directory and append path to access python packages/scripts within directory
    '''
    if src in os.listdir(root_dir):
        src_dir = os.path.join(root_dir, src)
        sys.path.append(src_dir)
    return sys.path[-1]

def set_folder(root_dir:str, folder:str) -> str:
    '''
    Set the folder path based on the folder name
    '''
    folder_path = os.path.join(
        root_dir, folder) if folder in os.listdir(root_dir) else os.curdir
    return folder_path

def set_path(path:str, dirname:str) -> str:
    '''
    '''
    return os.path.join(path, dirname)

def unzip(inpath:str, outpath:str) -> None:
    zf = ZipFile(inpath, 'r')
    zf.extractall(outpath)
    zf.close()

def set_weights(labels):
    return when(labels == 0, zratio).otherwise(1*(1-zratio))

## Setup

In [65]:
root_dir = get_root_dir('src', 5)
src_dir = set_src(root_dir, 'src')
data_dir = set_folder(root_dir, 'data')
raw_data_dir = set_path(data_dir, 'raw')
interim_data_dir = set_path(data_dir, 'interim')
processed_data_dir = set_path(data_dir, 'processed')
figures_dir = set_folder(root_dir, 'figures')
features_dir = set_folder(root_dir, 'features')
models_dir = set_folder(root_dir, 'models')

# 1. Data

## Initiate Spark session

In [4]:
#If not exists create a spark session named Anomaly Detection where the master node is local
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("Anomaly Detection") \
    .getOrCreate()

In [5]:
spark.getActiveSession()

## Load

### Set path

In [6]:
train_path = set_path(processed_data_dir, 'ML-MATT-CompetitionQT1920_train_processed.csv')
test_path = set_path(processed_data_dir, 'ML-MATT-CompetitionQT1920_test_processed.csv')

### Load data

In [7]:
train_df = spark.read.option("header", True) \
                .option("delimiter", ',') \
                .option("inferSchema" , "true") \
                .csv(train_path)

test_df = spark.read.option("header", True) \
                .option("delimiter", ',') \
                .option("inferSchema" , "true") \
                .csv(test_path)

In [8]:
train_df.printSchema()

root
 |-- CellName: string (nullable = true)
 |-- PRBUsageUL: double (nullable = true)
 |-- PRBUsageDL: double (nullable = true)
 |-- meanThr_DL: double (nullable = true)
 |-- meanThr_UL: double (nullable = true)
 |-- maxThr_DL: double (nullable = true)
 |-- maxThr_UL: double (nullable = true)
 |-- meanUE_DL: double (nullable = true)
 |-- meanUE_UL: double (nullable = true)
 |-- maxUE_DL: double (nullable = true)
 |-- maxUE_UL: double (nullable = true)
 |-- Unusual: integer (nullable = true)



In [9]:
train_df.show(5)

+--------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+-------+
|CellName|PRBUsageUL|PRBUsageDL|meanThr_DL|meanThr_UL|maxThr_DL|maxThr_UL|meanUE_DL|meanUE_UL|maxUE_DL|maxUE_UL|Unusual|
+--------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+-------+
|   3BLTE|    11.642|     1.393|      0.37|     0.041|   15.655|    0.644|    1.114|    1.025|     4.0|     3.0|      1|
|   1BLTE|    21.791|     1.891|     0.537|     0.268|   10.273|    1.154|    1.353|    1.085|     6.0|     4.0|      1|
|   9BLTE|     0.498|     0.398|     0.015|      0.01|    0.262|    0.164|    0.995|    0.995|     1.0|     1.0|      1|
|   4ALTE|     1.891|     1.095|      0.94|     0.024|   60.715|    0.825|    1.035|    0.995|     2.0|     2.0|      1|
|  10BLTE|     0.303|     0.404|     0.016|     0.013|    0.348|    0.168|    1.011|    1.011|     2.0|     1.0|      0|
+--------+----------+----------+

# 2. Feature Engineering

Because we have:

 - unbalanced sample
 - different scales 
 
we need to implement some transformations:

 - balance the train sample with weights
 - standardize the data
 
To do that let's create a Pipeline!


## Balancing Target

There are different methods to balance data:
  1. Undersampling (the majority class)
  2. Oversampling (the minority class) 
  3. Class weighting (assign the inverse ratio of each class as weights)

Although the sample is large (Undersampling is possible), I prefer Oversampling!

In [27]:
df_major_label = train_df.filter(col("Unusual") == 0)
df_minor_label= train_df.filter(col("Unusual") == 1)
ratio = int(df_major_label.count()/df_minor_label.count())
print("ratio: {}".format(ratio))

ratio: 2


In [48]:
range_ratio = range(ratio)
oversample_df = df_minor_label.withColumn("Dummy", explode(array([lit(i) for i in range_ratio]))).drop('Dummy') #Explode creates new row for each element in the array 

In [49]:
oversample_df.show(10)

+--------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+-------+
|CellName|PRBUsageUL|PRBUsageDL|meanThr_DL|meanThr_UL|maxThr_DL|maxThr_UL|meanUE_DL|meanUE_UL|maxUE_DL|maxUE_UL|Unusual|
+--------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+-------+
|   3BLTE|    11.642|     1.393|      0.37|     0.041|   15.655|    0.644|    1.114|    1.025|     4.0|     3.0|      1|
|   3BLTE|    11.642|     1.393|      0.37|     0.041|   15.655|    0.644|    1.114|    1.025|     4.0|     3.0|      1|
|   1BLTE|    21.791|     1.891|     0.537|     0.268|   10.273|    1.154|    1.353|    1.085|     6.0|     4.0|      1|
|   1BLTE|    21.791|     1.891|     0.537|     0.268|   10.273|    1.154|    1.353|    1.085|     6.0|     4.0|      1|
|   9BLTE|     0.498|     0.398|     0.015|      0.01|    0.262|    0.164|    0.995|    0.995|     1.0|     1.0|      1|
|   9BLTE|     0.498|     0.398|

In [69]:
train_df_balanced = df_major_label.unionAll(oversample_df).orderBy(rand())

In [70]:
ratio_balanced = train_df_balanced.where('Unusual == 0').count()/train_df_balanced.where('Unusual == 1').count()
print(f'The ratio now is {int(ratio_balanced)}')

The ratio now is 1


## Standardize data

### Train set

In [71]:
continuous_input_vars = ['PRBUsageUL', 'PRBUsageDL', 'meanThr_DL', 
                         'meanThr_UL', 'maxThr_DL', 'maxThr_UL', 
                         'meanUE_DL', 'meanUE_UL', 'maxUE_DL', 'maxUE_UL']

assembler = VectorAssembler(inputCols=continuous_input_vars,
                                   outputCol="features")

train_df_feat = assembler.transform(train_df_balanced)
train_df_feat.show(5)

+--------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+-------+--------------------+
|CellName|PRBUsageUL|PRBUsageDL|meanThr_DL|meanThr_UL|maxThr_DL|maxThr_UL|meanUE_DL|meanUE_UL|maxUE_DL|maxUE_UL|Unusual|            features|
+--------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+-------+--------------------+
|   7VLTE|     0.606|     1.516|     0.149|     0.014|   16.116|    1.119|    1.071|     0.01|     3.0|     2.0|      0|[0.606,1.516,0.14...|
|   6ULTE|     3.483|    14.328|     1.792|     0.081|   14.384|    2.295|    1.264|     0.01|     4.0|     4.0|      1|[3.483,14.328,1.7...|
|  10CLTE|     3.638|     0.606|      0.11|     0.019|   14.086|    0.567|    1.071|    1.021|     4.0|     2.0|      0|[3.638,0.606,0.11...|
|   6ALTE|     6.972|     1.415|     0.434|     0.049|   19.952|    0.749|    1.091|     0.01|     4.0|     3.0|      0|[6.972,1.415,0.43...|
|   6V

In [72]:
scaler = StandardScaler(withMean=True, withStd=True, inputCol=assembler.getOutputCol(), outputCol="features_std")
scaler_fit = scaler.fit(train_df_feat)
train_df_scaled = scaler_fit.transform(train_df_feat)
train_df_scaled = train_df_scaled.select("CellName", "features_std", "Unusual")
train_df_scaled.show(5)

+--------+--------------------+-------+
|CellName|        features_std|Unusual|
+--------+--------------------+-------+
|   7VLTE|[-0.8474628563740...|      0|
|   6ULTE|[-0.4931119676639...|      1|
|  10CLTE|[-0.4740211133886...|      0|
|   6ALTE|[-0.0633829962667...|      0|
|   6VLTE|[-0.6234224439431...|      0|
+--------+--------------------+-------+
only showing top 5 rows



### Test set

In [73]:
test_df_feat = assembler.transform(test_df)
test_df_feat.show(5)

+--------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+--------------------+
|CellName|PRBUsageUL|PRBUsageDL|meanThr_DL|meanThr_UL|maxThr_DL|maxThr_UL|meanUE_DL|meanUE_UL|maxUE_DL|maxUE_UL|            features|
+--------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+--------------------+
|   6ALTE|     3.781|     1.493|     0.575|     0.042|   22.659|    0.743|    0.985|     0.01|     3.0|     2.0|[3.781,1.493,0.57...|
|   6ULTE|     2.021|     3.335|     0.569|     0.075|   29.265|    1.049|    1.314|     0.01|     6.0|     3.0|[2.021,3.335,0.56...|
|   2ALTE|     0.505|     0.404|     0.014|      0.01|    0.227|    0.097|    1.011|     0.01|     2.0|     1.0|[0.505,0.404,0.01...|
|   3CLTE|     1.011|     0.505|     0.238|     0.021|   20.962|    0.609|    1.011|    1.011|     2.0|     1.0|[1.011,0.505,0.23...|
|   6CLTE|     3.881|     0.498|     0.076|     0.041|    3.93

In [74]:
test_df_scaled = scaler_fit.transform(test_df_feat)
test_df_scaled = test_df_scaled.select("CellName", "features_std")
test_df_scaled.show(5)

+--------+--------------------+
|CellName|        features_std|
+--------+--------------------+
|   6ALTE|[-0.4564082607346...|
|   6ULTE|[-0.6731818318607...|
|   2ALTE|[-0.8599027033534...|
|   3CLTE|[-0.7975803016546...|
|   6CLTE|[-0.4440915805569...|
+--------+--------------------+
only showing top 5 rows



## Store Features

In [75]:
# suppose to work
# scaler_fit.write().overwrite().save(features_dir)
train_features_path = set_path(interim_data_dir, 'ML-MATT-CompetitionQT1920_train_features.csv')
test_features_path = set_path(interim_data_dir, 'ML-MATT-CompetitionQT1920_test_features.csv')
train_df_scaled.toPandas().to_csv(train_features_path, index=False)
test_df.toPandas().to_csv(test_features_path, index=False)

# Conclusion

Now we get features. We are ready to train the model.